# Парсинг тайги
достаём всё про глаголы имеющие прямое и косвенное дополнения

### Импорты и загрузка текстов

In [ ]:
from operator import attrgetter
from pathlib import Path
from itertools import chain
from collections import Counter, namedtuple
import re
import json

from tqdm.auto import tqdm
import pandas as pd

!pip install conllu
import conllu

from typing import List, Dict

### Тайга сырая

In [ ]:
%%time
!tar -xzf './drive/My Drive/dir-indir/datasets/social.tar.gz'

CPU times: user 176 ms, sys: 26.8 ms, total: 203 ms
Wall time: 1min 10s


In [ ]:
%%time
!unzip -q './drive/My Drive/dir-indir/datasets/news.zip'

CPU times: user 667 ms, sys: 81.7 ms, total: 749 ms
Wall time: 4min 14s


In [ ]:
def get_filesize(path: Path, more_div=1):
    """return filesize
    default: div = 2^10 (kb)"""
    return int(path.stat().st_size / 2**10 / more_div)

In [ ]:
social_src = Path('home/tsha/social/tagged_texts')

In [ ]:
social = list(social_src.glob('*.txt'))
# kb-based total size of files
social_filesize = round(sum([file.stat().st_size / 2**10 for file in social]) + 1)

In [ ]:
news_srcs = ['Fontanka', 'Interfax', 'KP', 'Lenta']

news = iter([])
b = []
for news_src in news_srcs:
    news_src_tagged_texts = Path(news_src + '/' + 'texts_tagged')
    print(news_src_tagged_texts)
    # b.append(news_src_tagged_texts.glob('**/*.txt'))
    news = chain(news, news_src_tagged_texts.glob('**/*.txt'))

Fontanka/texts_tagged
Interfax/texts_tagged
KP/texts_tagged
Lenta/texts_tagged


In [ ]:
news = [new for new in news if 'metadata' not in str(new)]

In [ ]:
news_filesize = round(sum([file.stat().st_size / 2**10 for file in news]) + 1)

## парсинг размеченных текстов

### проверка

In [ ]:
s="""1	Я	я	PRON	_	Case=Nom|Number=Sing|Person=1	2	nsubj	_	TokenRange=206:207
2	дал	дать	VERB	_	Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	TokenRange=208:211
3	Пете	Пете	PROPN	_	Animacy=Anim|Case=Dat|Gender=Masc|Number=Sing	2	iobj	_	TokenRange=212:216
4	много	много	NUM	_	Animacy=Inan|Case=Acc	5	nummod:gov	_	TokenRange=217:222
5	денег	деньги	NOUN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Plur	2	obj	_	SpaceAfter=No|TokenRange=223:228
6	.	.	PUNCT	_	_	2	punct	_	SpacesAfter=No|TokenRange=228:229
""".replace("  ", "\t")

In [ ]:
s1 = """
1	По	по	ADP	_	_	2	case	_	_
2	словам	слово	NOUN	_	Animacy=Inan|Case=Dat|Gender=Neut|Number=Plur	7	parataxis	_	_
3	Поляковой	Полякова	PROPN	_	Animacy=Anim|Case=Gen|Gender=Fem|Number=Sing	2	nmod	_	SpaceAfter=No
4	,	,	PUNCT	_	_	3	punct	_	_
5	его	он	PRON	_	Case=Acc|Gender=Masc|Number=Sing|Person=3	7	obj	_	_
6	незаконно	незаконне	ADV	_	Degree=Pos	7	advmod	_	_
7	призвали	призывать	VERB	_	Aspect=Perf|Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	_
8	в	в	ADP	_	_	9	case	_	_
9	армию	армия	NOUN	_	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing	7	obl	_	SpaceAfter=No
10	,	,	PUNCT	_	_	9	punct	_	_
11	несмотря	несмотря	ADV	_	Polarity=Neg	13	case	_	_
12	на	на	ADP	_	_	11	fixed	_	_
13	то	то	PRON	_	Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing	7	obl	_	SpaceAfter=No
14	,	,	PUNCT	_	_	13	punct	_	_
15	что	что	SCONJ	_	_	13	mark	_	_
16	он	он	PRON	_	Case=Nom|Gender=Masc|Number=Sing|Person=3	17	nsubj	_	_
17	страдает	страдавать	VERB	_	Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act	13	advcl	_	_
18	серьезным	серьезный	ADJ	_	Case=Ins|Degree=Pos|Gender=Neut|Number=Sing	19	amod	_	_
19	заболеванием	заболевание	NOUN	_	Animacy=Inan|Case=Ins|Gender=Neut|Number=Sing	17	obl	_	_
20	крови	кровь	NOUN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing	19	nmod	_	SpaceAfter=No
21	,	,	PUNCT	_	_	20	punct	_	_
22	что	что	PRON	_	Case=Nom	20	nsubj	_	_
23	является	являться	VERB	_	Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Mid	0	conj	_	_
24	основанием	основание	NOUN	_	Animacy=Inan|Case=Ins|Gender=Neut|Number=Sing	23	obl	_	_
25	для	для	ADP	_	_	26	case	_	_
26	освобождения	освобождение	NOUN	_	Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing	24	nmod	_	_
27	от	от	ADP	_	_	28	case	_	_
28	прохождения	прохождение	NOUN	_	Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing	26	nmod	_	_
29	военной	военный	ADJ	_	Case=Gen|Degree=Pos|Gender=Fem|Number=Sing	30	amod	_	_
30	службы	служба	NOUN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing	28	nmod	_	SpaceAfter=No
31	.	.	PUNCT	_	_	30	punct	_	_""".replace("  ", "\t")

In [ ]:
s2 = """
1	Тем	то	PRON	_	Animacy=Inan|Case=Ins|Gender=Neut|Number=Sing	7	discourse	_	_
2	не	не	PART	_	Polarity=Neg	1	fixed	_	_
3	менее	менее	ADV	_	Degree=Cmp	2	fixed	_	SpaceAfter=No
4	,	,	PUNCT	_	_	3	punct	_	_
5	военно-врачебная	военно-врачебный	ADJ	_	Case=Nom|Degree=Pos|Gender=Fem|Number=Sing	6	amod	_	_
6	комиссия	комиссия	NOUN	_	Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing	7	nsubj	_	_
7	признала	признавать	VERB	_	Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	_
8	Романа	Роман	PROPN	_	Animacy=Anim|Case=Acc|Gender=Masc|Number=Sing	7	obj	_	_
9	Рудакова	Рудаков	PROPN	_	Animacy=Anim|Case=Acc|Gender=Masc|Number=Sing	8	flat:name	_	_
10	"	"	PUNCT	_	_	11	punct	_	SpaceAfter=No
11	годным	годный	ADJ	_	Case=Ins|Degree=Pos|Gender=Masc|Number=Sing	7	amod	_	SpaceAfter=No
12	"	"	PUNCT	_	_	11	punct	_	SpaceAfter=No
13	,	,	PUNCT	_	_	11	punct	_	_
14	и	и	CCONJ	_	_	16	cc	_	_
15	военкомат	военкомат	NOUN	_	Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing	16	nsubj	_	_
16	отправил	отправлять	VERB	_	Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	7	conj	_	_
17	его	он	PRON	_	Case=Acc|Gender=Masc|Number=Sing|Person=3	18	obj	_	_
18	служить	служить	VERB	_	Aspect=Imp|VerbForm=Inf|Voice=Act	16	xcomp	_	_
19	в	в	ADP	_	_	21	case	_	_
20	воинскую	воинский	ADJ	_	Case=Acc|Degree=Pos|Gender=Fem|Number=Sing	21	amod	_	_
21	часть	часть	NOUN	_	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing	18	obl	_	_
22	в	в	ADP	_	_	23	case	_	_
23	Каменке	Каменк	PROPN	_	Animacy=Inan|Case=Loc|Gender=Masc|Number=Sing	21	nmod	_	SpaceAfter=No
24	.	.	PUNCT	_	_	23	punct	_	_"""

In [ ]:
s_ioRelcl = """
1	Эти	этот	DET	_	Case=Acc|Number=Plur	2	amod	_	_
2	данные	данные	NOUN	_	Animacy=Inan|Case=Acc|Number=Plur	5	obj	_	_
3	им	они	PRON	_	Case=Dat|Number=Plur|Person=3	5	obl	_	_
4	были	быть	AUX	_	Aspect=Imp|Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Act	5	aux:pass	_	_
5	получены	получать	VERB	_	Aspect=Perf|Number=Plur|Tense=Past|Variant=Short|VerbForm=Part|Voice=Pass	0	root	_	_
6	от	от	ADP	_	_	8	case	_	_
7	российского	российский	ADJ	_	Case=Gen|Degree=Pos|Gender=Masc|Number=Sing	8	amod	_	_
8	консула	консул	NOUN	_	Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing	5	iobj	_	_
9	Олега	олег	PROPN	_	Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing	8	appos	_	_
10	Терентьева	Терентьев	PROPN	_	Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing	9	flat:name	_	SpaceAfter=No
11	,	,	PUNCT	_	_	10	punct	_	_
12	который	который	PRON	_	Case=Nom	13	nsubj	_	_
13	возглавляет	возглавлять	VERB	_	Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act	8	acl:relcl	_	_
14	оперативный	оперативный	ADJ	_	Animacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|Number=Sing	15	amod	_	_
15	штаб	штаб	NOUN	_	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing	13	obj	_	SpaceAfter=No
16	,	,	PUNCT	_	_	15	punct	_	_
17	созданный	создавать	VERB	_	Animacy=Inan|Aspect=Perf|Case=Acc|Gender=Masc|Number=Sing|Tense=Past|VerbForm=Part|Voice=Pass	15	amod	_	_
18	в	в	ADP	_	_	21	case	_	_
19	связи	связь	NOUN	_	Animacy=Inan|Case=Loc|Gender=Fem|Number=Sing	18	fixed	_	_
20	с	с	ADP	_	_	19	fixed	_	_
21	ДТП	дтп	PROPN	_	Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing	17	obl	_	SpaceAfter=No
22	.	.	PUNCT	_	_	21	punct	_	_""".replace('iobj', 'obl')

In [ ]:
s_obl_loc = """1	В	в	ADP	_	_	3	case	_	_
2	региональных	региональный	ADJ	_	Case=Loc|Degree=Pos|Number=Plur	3	amod	_	_
3	парламентах	парламент	NOUN	_	Animacy=Inan|Case=Loc|Gender=Masc|Number=Plur	6	obl	_	_
4	и	и	CCONJ	_	_	5	cc	_	_
5	СМИ	сми	PROPN	_	Animacy=Inan|Case=Nom|Gender=Neut|Number=Plur	6	nsubj	_	_
6	обсуждают	обсуждать	VERB	_	Aspect=Imp|Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act	0	root	_	_
7	процедуру	процедура	NOUN	_	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing	6	obj	_	_
8	изъятия	изъятие	NOUN	_	Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing	7	nmod	_	_
9	частной	частный	ADJ	_	Case=Gen|Degree=Pos|Gender=Fem|Number=Sing	10	amod	_	_
10	собственности	собственность	NOUN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing	8	nmod	_	_
11	и	и	CCONJ	_	_	12	cc	_	_
12	арендуемых	арендуть	VERB	_	Aspect=Imp|Case=Gen|Number=Plur|Tense=Pres|VerbForm=Part|Voice=Pass	8	conj	_	_
13	у	у	ADP	_	_	14	case	_	_
14	государства	государство	NOUN	_	Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing	12	obl	_	_
15	земель	земля	NOUN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Plur	14	nmod	_	SpaceAfter=No
16	.	.	PUNCT	_	_	15	punct	_	_
"""

In [ ]:
fields = ['lemma', 'upos', 'upos&feats', 'head&deprel']

In [ ]:
parse_sent(s_obl_loc, 'long', find_obl=True, exclude_loc=False,
    chain_tags=fields
)[0][0]._asdict()

OrderedDict([('text', 'long'),
             ('full_analysis',
              '1\tВ\tв\tADP\t_\t_\t3\tcase\t_\t_\n2\tрегиональных\tрегиональный\tADJ\t_\tCase=Loc|Degree=Pos|Number=Plur\t3\tamod\t_\t_\n3\tпарламентах\tпарламент\tNOUN\t_\tAnimacy=Inan|Case=Loc|Gender=Masc|Number=Plur\t6\tobl\t_\t_\n4\tи\tи\tCCONJ\t_\t_\t5\tcc\t_\t_\n5\tСМИ\tсми\tPROPN\t_\tAnimacy=Inan|Case=Nom|Gender=Neut|Number=Plur\t6\tnsubj\t_\t_\n6\tобсуждают\tобсуждать\tVERB\t_\tAspect=Imp|Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act\t0\troot\t_\t_\n7\tпроцедуру\tпроцедура\tNOUN\t_\tAnimacy=Inan|Case=Acc|Gender=Fem|Number=Sing\t6\tobj\t_\t_\n8\tизъятия\tизъятие\tNOUN\t_\tAnimacy=Inan|Case=Gen|Gender=Neut|Number=Sing\t7\tnmod\t_\t_\n9\tчастной\tчастный\tADJ\t_\tCase=Gen|Degree=Pos|Gender=Fem|Number=Sing\t10\tamod\t_\t_\n10\tсобственности\tсобственность\tNOUN\t_\tAnimacy=Inan|Case=Gen|Gender=Fem|Number=Sing\t8\tnmod\t_\t_\n11\tи\tи\tCCONJ\t_\t_\t12\tcc\t_\t_\n12\tарендуемых\tарендуть\tVERB\t_\tAspect=

### основной код

In [ ]:
sent_pattern = re.compile(
    '^# sent_id = (?P<sent_idx>\d+)\n'
    '# text = (?P<sent_text>.*?)\n'        # full sentence
    '(?P<analysis>.*?)' 
    '(?='                                  # lookahead: only take full analyses
        '\n(?:^# newpar$\n)?^# sent_id = (?:\d+)'
    ')', 
    
    flags=re.MULTILINE | re.DOTALL
)

sent_pattern2 = re.compile(
    '^# sent_id = (?P<sent_idx>\d+)\n'
    '# text = (?P<sent_text>(?:(?!\n1).)*?)\n'     # full sentence until analysis row 1
    '(?P<analysis>(?:(?!\n# sent_id).)*?Acc.*?)'   # full analysis featuring Acc
    '(?='                                          # lookahead: only take full analyses
        '\n(?:^# newpar$\n)?^# sent_id = (?:\d+)'
    ')', 
    
    flags=re.MULTILINE | re.DOTALL
)

In [ ]:
def utf8len(s):
    return len(s.encode('utf-8'))

In [ ]:
def feats_dict_to_str(feats_dict):
    return '|'.join([f'{feat}={val}' for feat, val in feats_dict.items()])

In [ ]:
def get_tag_single_token(token: conllu.models.Token, field: str,
    field_no_res_marker='?', child_field_marker='__') -> str:

    try:
        if child_field_marker in field:
            field, subfield = field.split(child_field_marker, maxsplit=1)
            res = token[field][subfield]
        else:
            res = token[field]
            if field == 'feats':
                res = feats_dict_to_str(res)
    except (TypeError, KeyError, AttributeError):
        res = field_no_res_marker

    return str(res)

In [ ]:
tok = {'feats': {'Asp': 'Impfv', 'Num': 'pl', 'wow': 'm'}, 'Mood': 'Ind'}
get_tag_single_token(tok, 'feats__wow')

'm'

In [ ]:
def get_token_fields_chains(
    tokens: conllu.TokenList, fields: List[str],
    token_sep = ' ', in_token_subfield_sep = '&',
    token_no_res_marker = '0',
    field_no_res_marker='?', child_field_marker='__',
) -> Dict[str, str]:
    get_tag_single_token_kwargs = dict(
        field_no_res_marker=field_no_res_marker,
        child_field_marker=child_field_marker)

    all_fields_chains = {}

    for field in fields:
        field_chain = ''
        for token in tokens:
            single_token_field_res = ''
            
            if in_token_subfield_sep in field:
                subfields = field.split(in_token_subfield_sep)
                for subfield in subfields:
                    single_token_field_res += (get_tag_single_token(
                            token, subfield, **get_tag_single_token_kwargs
                        ) + in_token_subfield_sep
                    )
                
                if single_token_field_res == (
                    (f'{field_no_res_marker}{in_token_subfield_sep}'
                    * len(subfields)).rstrip(in_token_subfield_sep)
                ):
                    single_token_field_res = token_no_res_marker

            else:
                single_token_field_res += get_tag_single_token(
                    token, field, **get_tag_single_token_kwargs
                )
                if single_token_field_res == field_no_res_marker:
                    single_token_field_res = token_no_res_marker

            field_chain += single_token_field_res.rstrip(
                in_token_subfield_sep) + token_sep

        all_fields_chains[field] = field_chain.rstrip(token_sep)
    
    return all_fields_chains


In [ ]:
def filter_objs(sent_tokens, objs, iobjs, oblobjs, find_obl, exclude_loc):
    skip_sentence = False
    # if not (len(objs) >= 1 and len(iobjs) >= 1):
    #     # if not(len(objs) >= 1 and len(oblobjs) >= 1):
    #     continue
    tree = sent_tokens.serialize
    if len(objs) >= 1:  # TODO: should we loook for objs someway else beforehand?
        if len(iobjs) == 0:
            # try iobjs dependent on direct obj
            obj = objs[0]
            i_fromo_objs = sent_tokens.filter(head=obj['id'], deprel='iobj')
            if len(i_fromo_objs) >= 1:
                # print(f'extending with i_fromo_objs (there are {len(i_fromo_objs)})\nsent is:\n{tree()}')
                iobjs.extend(i_fromo_objs)
            else:
                # # try oblobjs
                if find_obl and len(oblobjs) >= 1:
                    # print(f'extending with oblobj (there are {len(oblobjs)})\nsent is:\n{tree()}')
                    iobjs.extend(oblobjs)
                else:
                    skip_sentence = True
    else:
        skip_sentence = True

    if not skip_sentence and exclude_loc:
        iobj = iobjs[0]
        io_group_tokens, _ = parse_recursive_deps(sent_tokens, [iobj])
        for tok in io_group_tokens:
            try:
                if tok['feats']['Case'] == 'Loc':
                    skip_sentence = True
                    break
            except (TypeError, KeyError, AttributeError) as e:
                continue

    return skip_sentence, objs, iobjs, oblobjs


In [ ]:
def parse_recursive_deps(sentence, tokens_nohead_group, max_depth=3):
    all_tokens = list(tokens_nohead_group)
    has_relcl = ['0']

    cur_depth = 0

    def parse_(sentence, dependents, cur_depth):
        if cur_depth >= max_depth:
            return

        new_tokens = []
        for token in dependents:
            new_tokens.extend(
                [dependent_tok for dependent_tok in sentence.filter(head=token['id'])])
            if 'relcl' in token['deprel']:
                has_relcl[0] = '1'

        all_tokens.extend(new_tokens)
        parse_(sentence, new_tokens, cur_depth+1)
    
    parse_(sentence, tokens_nohead_group, 0)

    return all_tokens, has_relcl[0]

In [ ]:
def parse_sent(
    sent: str, text: str, find_obl=False, exclude_loc=False,
    dependents_max_depth=3,
    in_cell_sep=' ',
    verb_to_print=None,
    FEATURES=['Animacy', 'Case', 'Number', 'feats'],
    chain_tags=['lemma', 'upos', 'upos&feats'],
    get_token_fields_chains_kwargs={},
):
    sent_tokens = conllu.parse(sent)[0]  # [0] since we have one sentence always

    res = []
    # do_print = False
    print_filename = False
    times_printed = 0


    verbs = sent_tokens.filter(upos='VERB', feats__Voice='Act')
    for verb in verbs:

        # if verb_to_print:
        #     if verb['lemma'].lower() == verb_to_print:
        #         print(text, end='\n')
        #         times_printed += 1

        #         objs = sent_tokens.filter(head=verb['id'], deprel='obj')
        #         iobjs = sent_tokens.filter(head=verb['id'], deprel='iobj')
        #         if not (len(objs) >= 1 and len(iobjs) >= 1):
        #             print('not included', end='\n\n')
        #             if text == 'Я вам уже давно дала поручение касательно программы.':
        #                 print(sent)
        #         else:
        #             print('included', end='\n\n')
        #     continue

        # for every verb in sentence find respective objs and indirect objs
        objs = sent_tokens.filter(head=verb['id'], deprel='obj')
        iobjs = sent_tokens.filter(head=verb['id'], deprel='iobj')
        oblobjs = sent_tokens.filter(head=verb['id'], deprel='obl')
           
        skip_sentence, objs, iobjs, oblobjs = filter_objs(
            sent_tokens, objs, iobjs, oblobjs, find_obl, exclude_loc)
        if skip_sentence:
            continue

        has_many_objs = 0
        if len(objs) >= 2 or len(iobjs) >= 2:
            has_many_objs = 1
            # print(f'there are {len(objs)} objs and {len(iobjs)} iobjs '
            #       f'taking first '
            #       f"sent was:\n{text}\n{sent_tokens}\n")
        
        # take first element found and complete the obj and iobj phrases
        # TODO: optionally (?) exclude PUNCT from groups
        obj, iobj = objs[0], iobjs[0]
        objs_dir_deps = sent_tokens.filter(head=obj['id'])
        do_deps, do_has_relcl = parse_recursive_deps(
            sent_tokens, objs_dir_deps, max_depth=dependents_max_depth)
        objs_group = do_deps + [obj]
        objs_group_forms = ' '.join(
            [obj_dep['form'] for obj_dep in
             sorted(objs_group, key=lambda tok: tok['id'])]
        )

        iobjs_dir_deps = sent_tokens.filter(head=iobj['id'])
        io_deps, io_has_relcl = parse_recursive_deps(
            sent_tokens, iobjs_dir_deps, max_depth=dependents_max_depth)
        iobjs_group = io_deps + [iobj]
        iobjs_group_forms = ' '.join(
            [obj_dep['form'] for obj_dep in
             sorted(iobjs_group, key=lambda tok: tok['id'])]
        )

        # obj / iobj features
        objs_feats = {
            obj: {feat:'' for feat in FEATURES}
            for obj in ['obj', 'iobj']
        }
        for obj_type in ['obj', 'iobj']:
            for feat in FEATURES:                    
                cur_tok = obj if obj_type=='obj' else iobj
                try:
                    if feat == 'feats':
                        objs_feats[obj_type][feat] = feats_dict_to_str(cur_tok['feats'])
                    else:
                        objs_feats[obj_type][feat] = cur_tok['feats'][feat]
                except (TypeError, KeyError, AttributeError):  # when something strange is parsed as noun/pron and 
                    continue                                   #   either has no 'feats' or is None
        
        # check for negation (coded as "advmod" deprel, "PART" upos verb dependent)
        verb_neg = sent_tokens.filter(head=verb['id'], form='не', upos='PART', deprel='advmod')
        verb_has_neg = '1' if verb_neg else '0'
        
        # full verb features
        verb_feats = feats_dict_to_str(verb['feats']) if verb['feats'] else ''

        # adpositions are parsed as dependent on the noun, so we'll look for adp
        #   among iobjs_group to find them
        obj_nummod = sent_tokens.filter(head=obj['id'], deprel='nummod:gov')
        obj_head = obj_nummod[0]['form'].lower() if obj_nummod else verb['form'].lower()
        iobj_prep = sent_tokens.filter(head=iobj['id'], upos='ADP')
        iobj_head = iobj_prep[0]['form'].lower() if iobj_prep else verb['form'].lower()
        iobj_head_pos = 'ADP' if iobj_prep else 'VERB'

        # save order of objects and verb
        id_to_role = {obj['id']: 'DO', verb['id']: 'V', iobj['id']: 'IO'}
        ord = sorted(
            [verb] + [obj, iobj],
            key=lambda token: token['id'],
        )

        roles_order = in_cell_sep.join(
            id_to_role[token['id']] for token in ord)

        # get str chains of tags:
        tag2chain = get_token_fields_chains(
            sent_tokens, chain_tags
        )

        verb_res = Res(
            text, sent, has_many_objs,
                *[tag2chain[tag] for tag in chain_tags],
            verb['lemma'].lower(), verb['form'].lower(),
            verb_feats,
            verb_has_neg,
            objs_group_forms, iobjs_group_forms,
            obj['form'].lower(),
            obj['upos'], obj_head, obj['lemma'].lower(),
            *[objs_feats['obj'][key] for key in FEATURES], obj['deprel'], do_has_relcl,
            iobj['form'].lower(),
            iobj['upos'], iobj_head, iobj_head_pos, iobj['lemma'].lower(),
            *[objs_feats['iobj'][key] for key in FEATURES], iobj['deprel'], io_has_relcl,
            roles_order
        )
        res.append(verb_res)
 
    # if do_print:
    #     for res_ in res:
    #         print(res_[2:], end='\n\n')

    return res, print_filename, times_printed

In [ ]:
def count_verbs(
    sent: str, verb_forms_to_exclude=['Part'],
    verb_lemmas_counter: Counter = None, verb_lemmas_by_verbform=None
):
    sent_tokens = conllu.parse(sent)[0]  # [0] since we have one sentence always

    res = []
    print_filename = False

    verbs = sent_tokens.filter(upos='VERB')
    for verb in verbs:
        try:
            # print(verb, *verb.items())
            verbform = verb['feats']['VerbForm']
            verb_lemma = verb['lemma'].lower()
            if verbform not in verb_forms_to_exclude:
                    verb_lemmas_counter[verb_lemma] += 1
                    verb_lemmas_by_verbform[verbform][verb_lemma] +=1
        except (TypeError, KeyError):  # как глагол размечено что-то странное у чего нет фич
            print(verb, verb.items(), verb['lemma'] in verb_lemmas_counter)
            continue

In [ ]:
def parse_file(
    path, append_res_to, chunk_size_lines=300, prbar=None,
    sent_pattern=sent_pattern2,
    count_verb_lemmas=False, count_verbs_kwargs={},
    verb=None, times_printed=None,
    parse_sent_kwargs={},
):
    seen_sents = set()  # indices of sentences already parsed

    times_printed = 0
    with open(path, 'r', encoding='utf-8') as f:
        # read file in chunks
        last_chunk = '***start***'
        while last_chunk != '':
            chunk = ''.join([f.readline() for i in range(chunk_size_lines)])
            search_in = ''.join([last_chunk, chunk])
            last_chunk = chunk

            sents = sent_pattern.finditer(search_in)
            for i, sent in enumerate(sents, 1):
                sent_idx = sent.group('sent_idx')
                
                if sent_idx in seen_sents:
                    # print(f"repeat, id {sent_idx}")
                    continue

                seen_sents.add(sent_idx)
                
                if count_verb_lemmas:
                    count_verbs(sent.group('analysis'), **count_verbs_kwargs)
                    continue

                examples, print_filename, printed = parse_sent(
                    sent.group('analysis'), sent.group('sent_text'),
                    verb_to_print=verb,
                    **parse_sent_kwargs
                )
                times_printed += printed


                append_res_to.extend(examples)
                if print_filename:
                    print(f"file is `{path}`")

            if prbar:
                chunk_size = round(utf8len(chunk) / 2**10)
                prbar.update(chunk_size)
                
    return times_printed

In [ ]:
desired_cols = [
    'text', 'full_analysis',            # full texts
    'has_many_objs',                    # whether sentence has many objs with same deprel
    'lemma_chain', 'upos_chain',
        'upos_feats_chain',
        'head_id_deprel_chain',
    'verb_inf', 'verb',
    'verb_feats',                       # full verb features string
    'verb_has_neg',                     # binary: 1 - there is a "ne", 0 - there isn't
    'DO_group', 'IO_group',             # full NPs/PPs. TODO: add relclauses
    'DO',                               # word forms
    'DO_pos', 'DO_head', 'DO_lemma',    # `DO_head` may be unnecessary
        'DO_animacy', 'DO_case',
        'DO_number',
    'DO_feats', 'DO_deprel', 'DO_has_relcl',
    'IO',
    'IO_pos', 'IO_head', 'IO_head_pos', 'IO_lemma',
        'IO_animacy', 'IO_case',
        'IO_number',
    'IO_feats', 'IO_deprel', 'IO_has_relcl',
    'order',                            # in terms of V/IO/DO
    ]
Res = namedtuple('res', desired_cols)

In [ ]:
result = []

In [ ]:
len(result)

0

In [ ]:
fields = 'lemma upos upos&feats head&deprel'.split()
kwargs = {'find_obl': True, 'exclude_loc': True, 'chain_tags': fields}

In [ ]:
# with tqdm(total=get_filesize(src)) as prbar:
with tqdm(total=news_filesize + social_filesize) as prbar:
    for file in (news + social):
        parse_file(file, result, chunk_size_lines=450, prbar=prbar,
                   parse_sent_kwargs=kwargs)

In [ ]:
def save(result):
    path = Path('./drive/My Drive/dir-indir/results/res.json')
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=0)

In [ ]:
save(result)

### Просмотр примеров на конкретные глаголы

In [ ]:
# with tqdm(total=get_filesize(src)) as prbar:
with tqdm(total= news_filesize + social_filesize) as prbar:
    for file in (news + social):
        parse_file(file, result, chunk_size_lines=150, prbar=prbar, 
                   verb='брать', count=150)

In [ ]:
# with tqdm(total=get_filesize(src)) as prbar:
count = 200
printed = 0
verb='давать'

with tqdm(total= news_filesize + social_filesize) as prbar:
    for file in (news + social):
        printed += parse_file(file, [], chunk_size_lines=150, prbar=prbar, 
                              verb=verb)
        if printed >= count:
            break

В большинстве случаев он остается в тени, давая другим людям появляться в новостях.
not included

Даешь нашу биржу!
not included

Идея строительства бизнес-центра была озвучена еще в 1993 году, - вышло распоряжение бывшего мэра Петербурга Анатолия Собчака, которое давало разрешение на проектирование и строительство нового биржевого комплекса на Васильевском острове.
not included

Губернатор дала указание подготовить до конца года проектно-сметную документацию на все объекты парка, а также документы на выделение 11 миллионов рублей на ПСД для того самого обшарпанного конюшенного корпуса.
not included

Как отмечается в сообщении "Гринписа", еще в 2002 году по инициативе экологов в городе появились первые площадки для раздельного сбора мусора, а в 2005 году губернатор Валентина Матвиенко дала указание внедрить систему раздельного сбора мусора повсеместно.
not included

В субботу губернатор Петербурга еще раз дала понять, что крупноформатной рекламы в городе больше быть не должно.
not incl

### подсчёт употреблений по глаголам

In [ ]:
verb_lemmas_counter = Counter()
verb_forms_to_exclude = ['Part']
verb_lemmas_by_verbform = {
    verbform: Counter()
    for verbform in ['Fin', 'Inf', 'Conv', 'Part'] 
    if verbform not in verb_forms_to_exclude
}
count_verbs_kwargs = dict(
    verb_lemmas_counter=verb_lemmas_counter,
    verb_lemmas_by_verbform=verb_lemmas_by_verbform,
    verb_forms_to_exclude=verb_forms_to_exclude)

In [ ]:
with tqdm(total= news_filesize + social_filesize) as prbar:
    for file in (news + social):
        parse_file(
            file, [], chunk_size_lines=150, prbar=prbar,
            count_verb_lemmas=True, count_verbs_kwargs=count_verbs_kwargs
        )

{'id': 4, 'form': 'и.о.', 'lemma': 'и', 'upos': 'VERB', 'xpos': None, 'feats': None, 'head': 7, 'deprel': 'amod', 'deps': None, 'misc': None} dict_items([('id', 4), ('form', 'и.о.'), ('lemma', 'и'), ('upos', 'VERB'), ('xpos', None), ('feats', None), ('head', 7), ('deprel', 'amod'), ('deps', None), ('misc', None)]) False
{'id': 26, 'form': 'А.О.', 'lemma': 'А', 'upos': 'VERB', 'xpos': None, 'feats': None, 'head': 16, 'deprel': 'conj', 'deps': None, 'misc': {'SpaceAfter': 'No'}} dict_items([('id', 26), ('form', 'А.О.'), ('lemma', 'А'), ('upos', 'VERB'), ('xpos', None), ('feats', None), ('head', 16), ('deprel', 'conj'), ('deps', None), ('misc', {'SpaceAfter': 'No'})]) False
{'id': 27, 'form': 'и.о.', 'lemma': 'и', 'upos': 'VERB', 'xpos': None, 'feats': None, 'head': 26, 'deprel': 'dep', 'deps': None, 'misc': None} dict_items([('id', 27), ('form', 'и.о.'), ('lemma', 'и'), ('upos', 'VERB'), ('xpos', None), ('feats', None), ('head', 26), ('deprel', 'dep'), ('deps', None), ('misc', None)]) Fa

In [ ]:
pd.DataFrame(verb_lemmas_by_verbform)

,Fin,Inf,Conv
предлагать,47433.0,3708.0,1031.0
позволять,27551.0,2555.0,314.0
сообщать,317007.0,3577.0,157.0
добавлять,36698.0,3244.0,3147.0
отсутствевать,484.0,NaN,NaN
...,...,...,...
едросить,NaN,NaN,1.0
вступвить,NaN,NaN,1.0
прикормыать,NaN,NaN,1.0
кивнвуть,NaN,NaN,1.0


## запись результатов

In [ ]:
ig = attrgetter(*['DO_pos', 'IO_pos'])

In [ ]:
propn_examples = []
non_propn_examples = []
propn_cnt = 20
non_propn_cnt = 20

for res in result:
    if 'PROPN' in ig(res):
        if len(propn_examples) < propn_cnt:
            propn_examples.append(res)
    else:
        if len(non_propn_examples) < non_propn_cnt:
            non_propn_examples.append(res)
    if len(propn_examples) == propn_cnt and len(non_propn_examples) == non_propn_cnt:
        break
        

In [ ]:
id = 12
ex = propn_examples[id]
propn_examples[id]._asdict()

OrderedDict([('text',
              'ОТПРАВЬТЕ ЗАЯВКУ С ПОМЕТКОЙ " ФОНТАНКА" И ВЫИГРАЙТЕ ОТДЫХ НА ГОРНОЛЫЖНОМ КУРОРТЕ ТАХКО В НОВОГОДНИЙ ОТПУСК!'),
             ('full_analysis',
              '1\tОТПРАВЬТЕ\tОтправить\tVERB\t_\tAspect=Imp|Mood=Imp|Number=Plur|Person=2|VerbForm=Fin|Voice=Act\t0\troot\t_\t_\n2\tЗАЯВКУ\tЗаявка\tPROPN\t_\tAnimacy=Inan|Case=Acc|Gender=Fem|Number=Sing\t1\tobj\t_\t_\n3\tС\tс\tADP\t_\t_\t4\tcase\t_\t_\n4\tПОМЕТКОЙ\tпометкая\tPROPN\t_\tAnimacy=Anim|Case=Ins|Gender=Fem|Number=Sing\t1\tobl\t_\t_\n5\t"\t"\tPUNCT\t_\t_\t6\tpunct\t_\t_\n6\tФОНТАНКА\tФонтанк\tPROPN\t_\tAnimacy=Inan|Case=Gen|Gender=Masc|Number=Sing\t4\tnmod\t_\tSpaceAfter=No\n7\t"\t"\tPUNCT\t_\t_\t6\tpunct\t_\t_\n8\tИ\tи\tCCONJ\t_\t_\t9\tcc\t_\t_\n9\tВЫИГРАЙТЕ\tВыиграть\tVERB\t_\tAspect=Imp|Mood=Imp|Number=Plur|Person=2|VerbForm=Fin|Voice=Act\t0\tconj\t_\t_\n10\tОТДЫХ\tотдых\tNOUN\t_\tAnimacy=Inan|Case=Acc|Gender=Masc|Number=Sing\t9\tobj\t_\t_\n11\tНА\tна\tADP\t_\t_\t13\tcase\t_\t_\n12\tГОРНОЛЫЖНОМ\tГ

In [ ]:
def get_value_str_for_weak_confidence_dedup(res_tuple: Res, fields: List[str]) -> str:
    value_str = ''

    do_io_pos = ['DO_pos', 'IO_pos']
    pos_getter = attrgetter(*do_io_pos)
    # do_pos, io_pos = pos_getter(res_tuple)
    
    fields_not_to_include = []
    for pos_field in do_io_pos:
        pos = getattr(res_tuple, pos_field, '')
        if pos == 'PROPN':
            obj_type = pos_field.split('_')[0]
            fields_not_to_include.extend(
                [
                 f'{obj_type}_lemma', f'{obj_type}',  # lemma and form
                 f'{obj_type}_feats',                 # includes gender that is different for male and female names
                ])
            
    for field in fields_not_to_include:
        try:
            fields.remove(field)
        except ValueError:
            continue

    for field in fields:
        value_str += getattr(res_tuple, field, '')

    return value_str

In [ ]:
def dedup_by_value(
    result_tuples: List[Res],
    fewer_fields_to_filter=None,
    value_starting_index=desired_cols.index('verb_inf'),
):
    """Deduplicate list of results
    For results that are hard duplicates - 
        everything concatenated (text and parsed data) is the same,
        we delete them alltogether
    For those that are possible duplicates (values are same),
        we mark them
    """
    seen_entries = set()
    seen_values_comb2id = {}
    seen_reduced_values_comb2id = {}
    value_cnt = 0
    reduced_value_cnt = 0

    deduplicated_res = []
    for res_tuple in result_tuples: 
        # everything: text + parsed data (except for full analysis):
        entry = '||'.join(res_tuple[0:1]+tuple(str(el) for el in res_tuple[value_starting_index:]))
        value = '||'.join(tuple(str(el) for el in res_tuple[value_starting_index:]))  

        is_pos_dupe = ('0',)  # adding bool field that marks possible dupes
        if entry not in seen_entries:
            seen_entries.add(entry)

            pos_weak_conf_dupe_id = ('', )
            if value in seen_values_comb2id:
                is_pos_dupe = ('1',)
            else:
                seen_values_comb2id[value] = value_cnt
                value_cnt += 1

                ### 
                if fewer_fields_to_filter:
                    reduced_value = get_value_str_for_weak_confidence_dedup(
                        res_tuple, fewer_fields_to_filter
                    )
                    if reduced_value in seen_reduced_values_comb2id:
                        is_pos_dupe = ('2',)
                    else:
                        seen_reduced_values_comb2id[reduced_value] = reduced_value_cnt
                        reduced_value_cnt += 1
                    
                    pos_weak_conf_dupe_id = (seen_reduced_values_comb2id[reduced_value], )
                

            pos_strong_conf_dupe_id = (seen_values_comb2id[value], )
            
            deduplicated_res.append(
                is_pos_dupe + pos_strong_conf_dupe_id + pos_weak_conf_dupe_id
                + res_tuple)
            
            del is_pos_dupe, pos_strong_conf_dupe_id, pos_weak_conf_dupe_id
        del res_tuple
    
    return deduplicated_res

In [ ]:
Res(*desired_cols)

res(text='text', full_analysis='full_analysis', has_many_objs='has_many_objs', lemma_chain='lemma_chain', upos_chain='upos_chain', upos_feats_chain='upos_feats_chain', head_id_deprel_chain='head_id_deprel_chain', verb_inf='verb_inf', verb='verb', verb_feats='verb_feats', verb_has_neg='verb_has_neg', DO_group='DO_group', IO_group='IO_group', DO='DO', DO_pos='DO_pos', DO_head='DO_head', DO_lemma='DO_lemma', DO_animacy='DO_animacy', DO_case='DO_case', DO_number='DO_number', DO_feats='DO_feats', DO_deprel='DO_deprel', DO_has_relcl='DO_has_relcl', IO='IO', IO_pos='IO_pos', IO_head='IO_head', IO_head_pos='IO_head_pos', IO_lemma='IO_lemma', IO_animacy='IO_animacy', IO_case='IO_case', IO_number='IO_number', IO_feats='IO_feats', IO_deprel='IO_deprel', IO_has_relcl='IO_has_relcl', order='order')

In [ ]:
class JSONResDecoder(json.JSONDecoder):
    def __init__(self, *args, **kwargs):
        json.JSONDecoder.__init__(self, object_hook=self.object_hook, *args, **kwargs)
    def object_hook(self, obj):
        if isinstance(obj, str):
            return obj
        elif any(isinstance(content, str) for content in obj):
            return Res
        else:
            return obj

In [ ]:
result = []
cols = len(Res(*desired_cols))
checkpoint = 3 * 10**5
i=0

In [ ]:
def save(obj, fn, path='./drive/My Drive/dir-indir/results/{}'):
    with open(path.format(fn), 'w', encoding='utf-8') as f:
        json.dump(obj, f)

In [ ]:
prbar = tqdm(total=9*10**5)
with open(path, 'r', encoding='utf-8') as f:
    f.readline()
    while f:
        if i % checkpoint == 0:
            save(result, f'resPart_{i // checkpoint}.json')
            result.clear()

        buf = []
        f.readline()
        for i in range(cols):
            line = f.readline()
            buf.append(line.strip().strip(',').strip('"'))
        f.readline()

        if all(val == '' for val in buf):
            break
        
        result.append(Res(*buf))
        
        prbar.update(1)

        i += 1

prbar.close()

In [ ]:
len(result)

10

In [ ]:
get_filesize(path)

NameError: ignored

In [ ]:
get_filesize()

In [ ]:
def write_csv(result_df, filename='out.csv'):
    filename = Path(filename)

    if filename.exists():
        raise ValueError

    result_df.to_csv(filename)

In [ ]:
fields = 'verb_inf verb verb_feats verb_has_neg DO DO_pos DO_head DO_lemma DO_animacy DO_case DO_number DO_deprel IO IO_pos IO_head IO_head_pos IO_lemma IO_animacy IO_case IO_number IO_deprel order'.split()
fields

['verb_inf',
 'verb',
 'verb_feats',
 'verb_has_neg',
 'DO',
 'DO_pos',
 'DO_head',
 'DO_lemma',
 'DO_animacy',
 'DO_case',
 'DO_number',
 'DO_deprel',
 'IO',
 'IO_pos',
 'IO_head',
 'IO_head_pos',
 'IO_lemma',
 'IO_animacy',
 'IO_case',
 'IO_number',
 'IO_deprel',
 'order']

In [ ]:
dupe_info_cols = 'pos_dupe pos_strong_dupe_id pos_weak_dupe_id'.split()

dres = dedup_by_value(result)
print(f'total results: {len(result)}, with simple dedup: {len(dres)}')

total results: 953863, with simple dedup: 845053


In [ ]:
df = pd.DataFrame(dres, columns=dupe_info_cols + desired_cols)
df.head()

,pos_dupe,pos_strong_dupe_id,pos_weak_dupe_id,text,full_analysis,has_many_objs,lemma_chain,upos_chain,upos_feats_chain,head_id_deprel_chain,verb_inf,verb,verb_feats,verb_has_neg,DO_group,IO_group,DO,DO_pos,DO_head,DO_lemma,DO_animacy,DO_case,DO_number,DO_feats,DO_deprel,DO_has_relcl,IO,IO_pos,IO_head,IO_head_pos,IO_lemma,IO_animacy,IO_case,IO_number,IO_feats,IO_deprel,IO_has_relcl,order
0,0,0,,После попытки военных KFOR разобрать баррикаду...,1\tПосле\tпосле\tADP\t_\t_\t2\tcase\t_\t_\n2\t...,0,после попытка военный Kfor разбирать баррикада...,ADP NOUN NOUN PROPN VERB NOUN NOUN VERB VERB P...,ADP&? NOUN&Animacy=Inan|Case=Gen|Gender=Fem|Nu...,2&case 5&obl 2&nmod 3&flat:foreign 8&nsubj 5&o...,разбирать,разобрать,Aspect=Perf|VerbForm=Inf|Voice=Act,0,баррикаду сербы,После попытки военных KFOR,баррикаду,NOUN,разобрать,баррикада,Inan,Acc,Sing,Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing,obj,0,попытки,NOUN,после,ADP,попытка,Inan,Gen,Sing,Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing,obl,0,IO V DO
1,0,1,,После попытки военных KFOR разобрать баррикаду...,1\tПосле\tпосле\tADP\t_\t_\t2\tcase\t_\t_\n2\t...,0,после попытка военный Kfor разбирать баррикада...,ADP NOUN NOUN PROPN VERB NOUN NOUN VERB VERB P...,ADP&? NOUN&Animacy=Inan|Case=Gen|Gender=Fem|Nu...,2&case 5&obl 2&nmod 3&flat:foreign 8&nsubj 5&o...,кидать,закидать,Aspect=Perf|VerbForm=Inf|Voice=Act,0,их,камнями .,их,PRON,закидать,они,,Acc,Plur,Case=Acc|Number=Plur|Person=3,obj,0,камнями,NOUN,закидать,VERB,камень,Inan,Ins,Plur,Animacy=Inan|Case=Ins|Gender=Masc|Number=Plur,obl,0,V DO IO
2,0,2,,Для разгона толпы военные вновь применили слез...,1\tДля\tдля\tADP\t_\t_\t2\tcase\t_\t_\n2\tразг...,0,для разгон толпа военный вновь применять слезо...,ADP NOUN NOUN NOUN ADV VERB ADJ NOUN PUNCT CCO...,ADP&? NOUN&Animacy=Inan|Case=Gen|Gender=Masc|N...,2&case 6&obl 2&nmod 6&nsubj 6&advmod 0&root 8&...,применять,применили,Aspect=Perf|Mood=Ind|Number=Plur|Tense=Past|Ve...,0,"слезоточивый газ ,",Для разгона толпы,газ,NOUN,применили,газ,Inan,Acc,Sing,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,obj,0,разгона,NOUN,для,ADP,разгон,Inan,Gen,Sing,Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing,obl,0,IO V DO
3,0,3,,"Как объяснили в пресс-службе ведомства, весной...",1\tКак\tкак\tADV\t_\tDegree=Pos\t12\tmark\t_\t...,0,"как объяснять в пресс-служба ведомство , весна...",ADV VERB ADP NOUN NOUN PUNCT NOUN ADV PUNCT AD...,ADV&Degree=Pos VERB&Aspect=Perf|Mood=Ind|Numbe...,12&mark 12&advcl 4&case 2&obl 4&nmod 5&punct 1...,проходить,прошли,Aspect=Perf|Mood=Ind|Number=Plur|Tense=Past|Ve...,0,"ярмарки вакансий "" .",весной,ярмарки,NOUN,прошли,ярмарка,Inan,Acc,Plur,Animacy=Inan|Case=Acc|Gender=Fem|Number=Plur,obj,0,весной,NOUN,прошли,VERB,весна,Inan,Ins,Sing,Animacy=Inan|Case=Ins|Gender=Fem|Number=Sing,obl,0,IO V DO
4,0,4,,К тому же часть работодателей предоставляют ме...,1\tК\tк\tADP\t_\t_\t2\tcase\t_\t_\n2\tтому\tто...,0,к то же часть работодатель предоставлять место...,ADP PRON PART NOUN NOUN VERB NOUN ADP NOUN PUN...,ADP&? PRON&Animacy=Inan|Case=Dat|Gender=Neut|N...,2&case 6&obl 2&advmod 6&obj 4&nmod 0&root 9&ob...,предоставлять,предоставляют,Aspect=Imp|Mood=Ind|Number=Plur|Person=3|Tense...,0,часть работодателей,К тому же,часть,NOUN,предоставляют,часть,Inan,Acc,Sing,Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing,obj,0,тому,PRON,к,ADP,то,Inan,Dat,Sing,Animacy=Inan|Case=Dat|Gender=Neut|Number=Sing,obl,0,IO DO V


In [ ]:
df[df['pos_dupe']=='2']

,pos_dupe,pos_strong_dupe_id,pos_weak_dupe_id,text,full_analysis,has_many_objs,lemma_chain,upos_chain,upos_feats_chain,head_id_chain,verb_inf,verb,verb_feats,verb_has_neg,DO_group,IO_group,DO,DO_pos,DO_head,DO_lemma,DO_animacy,DO_case,DO_number,DO_feats,DO_deprel,DO_has_relcl,IO,IO_pos,IO_head,IO_head_pos,IO_lemma,IO_animacy,IO_case,IO_number,IO_feats,IO_deprel,IO_has_relcl,order
158,2,158,130,51-летнего мужчину и 21-летнюю женщину достави...,1\t51-летнего\t51-летний\tADJ\t_\tAnimacy=Anim...,0,51-летний мужчина и 21-летний женщина доставля...,ADJ NOUN CCONJ ADJ NOUN VERB ADP ADJ NOUN PUNC...,ADJ&Animacy=Anim|Case=Acc|Degree=Pos|Gender=Ma...,2&amod 6&obj 5&cc 5&amod 2&conj 0&root 9&case ...,доставлять,доставили,Aspect=Perf|Mood=Ind|Number=Plur|Tense=Past|Ve...,0,51-летнего мужчину и 21-летнюю женщину,"в Александровскую больницу ,",мужчину,NOUN,доставили,мужчина,Anim,Acc,Sing,Animacy=Anim|Case=Acc|Gender=Masc|Number=Sing,obj,0,больницу,NOUN,в,ADP,больница,Inan,Acc,Sing,Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing,obl,0,DO V IO
201,2,201,195,Сейчас в доме разворачивается самая настоящая ...,1\tСейчас\tсейчас\tADV\t_\tDegree=Pos\t4\tadvm...,0,сейчас в дом развертываться самый настоящий во...,ADV ADP NOUN VERB ADJ ADJ NOUN PUNCT ADJ NOUN ...,ADV&Degree=Pos ADP&? NOUN&Animacy=Inan|Case=Lo...,4&advmod 3&case 4&obl 0&root 6&amod 7&amod 4&n...,переизбирать,переизбрать,Aspect=Perf|VerbForm=Inf|Voice=Act,0,председателя,"ТСЖ ,",председателя,NOUN,переизбрать,председатель,Anim,Acc,Sing,Animacy=Anim|Case=Acc|Gender=Masc|Number=Sing,obj,0,тсж,PROPN,переизбрать,VERB,тсж,Inan,Ins,Sing,Animacy=Inan|Case=Ins|Gender=Neut|Number=Sing,obl,0,V DO IO
238,2,238,235,Козак потер нос белому медведю,1\tКозак\tкозак\tNOUN\t_\tAnimacy=Anim|Case=No...,0,козак потирать нос белый медведь,NOUN VERB NOUN ADJ NOUN,NOUN&Animacy=Anim|Case=Nom|Gender=Masc|Number=...,2&nsubj 0&root 2&obj 5&amod 2&obl,потирать,потер,Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|T...,0,нос,белому медведю,нос,NOUN,потер,нос,Inan,Acc,Sing,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,obj,0,медведю,NOUN,потер,VERB,медведь,Anim,Dat,Sing,Animacy=Anim|Case=Dat|Gender=Masc|Number=Sing,obl,0,V DO IO
361,2,361,83,Фондовая биржа ММВБ начала торги среды ростом ...,1\tФондовая\tФондовой\tADJ\t_\tCase=Nom|Degree...,0,Фондовой биржа Ммвб начинать торг среда рост и...,ADJ NOUN PROPN VERB NOUN NOUN NOUN NOUN ADP NU...,ADJ&Case=Nom|Degree=Pos|Gender=Fem|Number=Sing...,2&amod 4&nsubj 2&nmod 0&root 4&obj 5&nmod 4&ob...,начинать,начала,Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Te...,0,торги среды,ростом индекса,торги,NOUN,начала,торг,Inan,Acc,Plur,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,obj,0,ростом,NOUN,начала,VERB,рост,Inan,Ins,Sing,Animacy=Inan|Case=Ins|Gender=Masc|Number=Sing,obl,0,V DO IO
373,2,373,240,"ООО ""Тамбов-цемент"", созданное под проект, име...",1\tООО\tОоо\tADV\t_\tDegree=Pos\t10\tadvmod\t_...,0,"Ооо "" Тамбов-цемент "" , создавать под проект ,...",ADV PUNCT PROPN PUNCT PUNCT VERB ADP NOUN PUNC...,ADV&Degree=Pos PUNCT&? PROPN&Animacy=Inan|Case...,10&advmod 3&punct 1&appos 3&punct 3&punct 3&am...,иметь,имеет,Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense...,0,лицензию,на разработку Борисовского месторождения извес...,лицензию,NOUN,имеет,лицензия,Inan,Acc,Sing,Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing,obj,0,разработку,NOUN,на,ADP,разработка,Inan,Acc,Sing,Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing,obl,0,V DO IO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8193,2,8142,2474,Игроки должны нести ответственность за свои де...,1\tИгроки\tигрок\tNOUN\t_\tAnimacy=Anim|Case=N...,0,игрок должен нести ответственность за свой дей...,NOUN ADJ VERB NOUN ADP DET NOUN PUNCT,NOUN&Animacy=Anim|Case=Nom|Gender=Masc|Number=...,2&nsubj 0&root 2&dep 3&obj 7&case 7&amod 3&obl...,нести,нести,Aspect=Imp|VerbForm=Inf|Voice=Act,0,ответственность,за свои действия .,ответственность,NOUN,нести,ответств

In [ ]:
pos_dupe_val = '2'
pos_dupe_col_name = 'pos_weak_dupe_id'

for row in df[df['pos_dupe']=='1'].itertuples():
    cur_id = row[0]
    dupe_id = getattr(row, pos_dupe_col_name)
    print(cur_id, dupe_id, df[df[pos_dupe_col_name] == dupe_id]['text'].values)

709  ['РТС получасом ранее начала торги на уровне закрытия пятницы, 10 июня, но к 10.30 мск индекс уже подрос на 0,06% - до 1927,22 пункта, сообщает РИА Новости.'
 'Из них в движении находятся 11 кораблей, 6 ждет разрешения на заход в порт, а 109 не могут сделать этого без ледокольной помощи.'
 'Из них в движении находятся 18 кораблей, 26 ждет разрешения на заход в порт, а 26 не могут сделать этого без ледокольной помощи.'
 'Универсальный вертолетоносец "Мистраль" водоизмещением 21 тысяча тонн и максимальной длиной корпуса 210 метров способен развивать скорость более 18 узлов.'
 '"Сегодня нам предлагают всей семьей собирать модели этих танков, а затем поставить их на полочку для восхищения этим шедевром немецкой техники.'
 'В начале сентября ПНС продлил срок ультиматума верным Каддафи отрядам, чтобы избежать кровопролития и разрушений, и начал вести с ними переговоры.'
 'Как говорят в Балтийской ассоциации защиты прав туристов, пассажиры имеют право требовать от перевозчика компенсацию

In [ ]:
df[df['verb_inf']=='ограждать']

,pos_dupe,text,full_analysis,verb_inf,verb,verb_feats,verb_has_neg,DO_group,IO_group,DO,DO_pos,DO_head,DO_lemma,DO_animacy,DO_case,DO_number,DO_feats,DO_deprel,DO_has_relcl,IO,IO_pos,IO_head,IO_lemma,IO_animacy,IO_case,IO_number,IO_feats,IO_deprel,IO_has_relcl,order
1627,0,Оппонентов оградили друг от друга.,1\tОппонентов\tоппонент\tNOUN\t_\tAnimacy=Anim...,ограждать,оградили,Aspect=Perf|Mood=Ind|Number=Plur|Tense=Past|Ve...,0,Оппонентов,друг от друга .,оппонентов,NOUN,оградили,оппонент,Anim,Acc,Plur,Animacy=Anim|Case=Acc|Gender=Masc|Number=Plur,obj,0,друга,NOUN,от,друг,Anim,Gen,Sing,Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing,iobj,0,DO; V; IO


In [ ]:
fn = 'taiga3.2_withObl_noLoc.сsv'
write_csv(df, fn)
write_csv(df, './drive/My Drive/dir-indir/results/'+fn)

In [ ]:
# dedup_objs_df = pd.read_csv('taiga2.csv', index_col=0)

In [ ]:
dedup_objs_df

,pos_dupe,text,full_analysis,verb_inf,verb,DO_group,IO_group,DO,DO_pos,DO_head,DO_lemma,IO,IO_pos,IO_head,IO_lemma,order
0,0,13 декабря его отправили с Ладожского вокзала ...,1\t13\t13\tNUM\t_\t_\t4\tnummod\t_\t_\n2\tдека...,отправлять,отправили,NaN,с Ладожского,его,PRON,отправили,он,вокзала,NOUN,с,вокзал,DO; V; IO
1,0,Собравшиеся потребовали выселения из города вы...,1\tСобравшиеся\tсобираться\tVERB\t_\tAspect=Pe...,потребовывать,потребовали,NaN,из выходцев,выселения,NOUN,потребовали,выселение,города,NOUN,из,город,V; DO; IO
2,0,"- до 100 л .с. (до 73,55 кВт) включительно 25 ...",1\t-\t-\tPUNCT\t_\t_\t2\tpunct\t_\t_\n2\tдо\tд...,ть,л,NaN,до 150 .с.,.с.,NOUN,л,.сяча,л,NOUN,до,литр,V; DO; IO
3,0,Наличные деньги доставлялись в петербургский ф...,1\tНаличные\tналичный\tADJ\t_\tCase=Nom|Degree...,доставлять,доставлялись,.,из головного,неделю,NOUN,доставлялись,неделя,офиса,NOUN,из,офис,V; IO; DO
4,0,"Человек просто испортил себе жизнь, и это траг...",1\tЧеловек\tчеловек\tNOUN\t_\tAnimacy=Anim|Cas...,испорчивать,испортил,",",NaN,жизнь,NOUN,испортил,жизнь,себе,PRON,испортил,себя,V; IO; DO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3166,0,Собрали петицию против несуществующего феромар...,1\tСобрали\tсобирать\tVERB\t_\tAspect=Perf|Moo...,собирать,Собрали,NaN,против несуществующего феромарганцевого .,петицию,NOUN,Собрали,петиция,завода,NOUN,против,завод,V; DO; IO
3167,0,"Им с этой ошибкой жить всю жизнь, им через при...",1\tИм\tони\tPRON\t_\tCase=Dat|Number=Plur|Pers...,жить,жить,"всю ,",NaN,жизнь,NOUN,жить,жизнь,Им,PRON,жить,они,IO; V; DO
3168,0,Уже после увольнения руководителей издания пре...,1\tУже\tуже\tADV\t_\tDegree=Pos\t3\tadvmod\t_\...,опровергать,опровергал,NaN,из,давление,NOUN,опровергал,давление,Кремля,PROPN,из,кремль,V; DO; IO
3169,0,"Что у нас в 90-х - бардак, ""кровавые мальчики""...",1\tЧто\tчто\tPRON\t_\tCase=Acc\t3\tnsubj\t_\t_...,разывать,развал,",",Что у 90-х,страны,NOUN,развал,страна,нас,PRON,у,мы,IO; V; DO


In [ ]:
objs_df_verb_lemmas = Counter()
for lemma in dedup_objs_df['verb_inf']:
    objs_df_verb_lemmas[lemma.lower()]+=1

In [ ]:
total_vs_2objs_verb_lemmas = {'total': [], 'double_objs_dedup': [], 'verb': []}

In [ ]:
for verb_lemma, count in verb_lemmas_counter.items():
    total_vs_2objs_verb_lemmas['total'].append(count)
    total_vs_2objs_verb_lemmas['double_objs_dedup'].append(objs_df_verb_lemmas[verb_lemma])
    total_vs_2objs_verb_lemmas['verb'].append(verb_lemma)

In [ ]:
comparison_total_2objs = pd.DataFrame(total_vs_2objs_verb_lemmas, )

In [ ]:
fn = 'verb_counts.csv'
# write_csv(comparison_total_2objs, fn)
write_csv(comparison_total_2objs, './drive/My Drive/dir-indir/results/'+fn)

In [ ]:
big_df = pd.read_csv('./drive/My Drive/dir-indir/results/taiga3.2_withObl_noLoc.сsv')

In [ ]:
fn = 'taiga3.2_withObl_noLoc_REDUCED.сsv'
fn2 = './drive/My Drive/dir-indir/results/' + fn

In [ ]:
[c for c in big_df.columns if c!='full_analysis']

['Unnamed: 0',
 'pos_dupe',
 'text',
 'has_many_objs',
 'verb_inf',
 'verb',
 'verb_feats',
 'verb_has_neg',
 'DO_group',
 'IO_group',
 'DO',
 'DO_pos',
 'DO_head',
 'DO_lemma',
 'DO_animacy',
 'DO_case',
 'DO_number',
 'DO_feats',
 'DO_deprel',
 'DO_has_relcl',
 'IO',
 'IO_pos',
 'IO_head',
 'IO_head_pos',
 'IO_lemma',
 'IO_animacy',
 'IO_case',
 'IO_number',
 'IO_feats',
 'IO_deprel',
 'IO_has_relcl',
 'order']

In [ ]:
big_df.to_csv(
    fn, columns=
    [
     'text', 'full_analysis', 
     'verb_inf', 'verb',
     'DO', 'IO', 'DO_pos', 'IO_pos',
     'IO_head', 'IO_head_pos',
     'order'],
)

In [ ]:
big_df.to_csv(
    'big.csv'
)


In [ ]:
def write_split_df(df, name_cols_dict, 
    path='./drive/My Drive/dir-indir/results/{name}{ext}',
    ext='.csv', index_label='index'):

    for name, cols in name_cols_dict.items():
        df.to_csv(
            path.format(name=name, ext=ext),
            columns=cols,
            index_label=index_label,
        )
        print(f'sucessfully written {path}'.format(name=name, ext=ext))

In [ ]:
def get_name_cols_dict(df, name):
    return {
        f'{name}_withObl_noLoc_fullAna':['full_analysis'],
        f'{name}_withObl_noLoc_forFiltering':[
            'text', 
            'verb_inf', 'verb',
            'DO', 'IO', 'DO_pos', 'IO_pos',
            'IO_head', 'IO_head_pos',
            'order'
        ],
        f'{name}_withObl_noLoc_fullNoAna':[
            c for c in df.columns if c!='full_analysis'
        ]
    }

In [ ]:
name_cols_dict = get_name_cols_dict(big_df[[
            c for c in big_df.columns if c!='Unnamed: 0'
        ]])

In [ ]:
name_cols_dict = get_name_cols_dict(df, 'taiga_2')

In [ ]:
name_cols_dict

In [ ]:
write_split_df(df, name_cols_dict)

sucessfully written ./drive/My Drive/dir-indir/results/taiga_2_withObl_noLoc_fullAna.csv
sucessfully written ./drive/My Drive/dir-indir/results/taiga_2_withObl_noLoc_forFiltering.csv
sucessfully written ./drive/My Drive/dir-indir/results/taiga_2_withObl_noLoc_fullExcepAna.csv


In [ ]:
!cd './drive/My Drive/dir-indir/results/'

In [ ]:
!ls -a

.  ..  .config	drive  sample_data


In [ ]:
dir = "./drive/My Drive/dir-indir/results/"
split_prog = dir + 'split.py'
inp_file = dir + 'taiga_2_withObl_noLoc_forFiltering.csv' 

!python {split_prog} {inp_file} 300000

python3: can't open file './drive/My': [Errno 2] No such file or directory
